<a href="https://colab.research.google.com/github/hollipista/MetaCortex-AAEE/blob/main/01_MetaCortex_AAEE_prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Lehet kihagyni lépéseket, ha nem elsőre futtatod és már megvannak a fájlok a környezetben,
# de ezt mindenképpen futtasd le - értelemszerűen.
import zipfile
import numpy as np
import pandas as pd
from io import StringIO
import re
import gc
import os
import zipfile

In [ ]:
!wget -O CRSP_data.zip https://www.dropbox.com/scl/fi/i3o5htygyu9c0be7outsb/CRSP_data.zip?rlkey=zclrhry1dhhid7lspoumkocer&dl=0

--2023-11-20 19:30:57--  https://www.dropbox.com/scl/fi/i3o5htygyu9c0be7outsb/CRSP_data.zip?rlkey=zclrhry1dhhid7lspoumkocer
Resolving www.dropbox.com (www.dropbox.com)... 162.125.85.18, 2620:100:6035:18::a27d:5512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.85.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ucdaf60311b0d0814212532a2830.dl.dropboxusercontent.com/cd/0/inline/CH5ZVA9DyVq4fWuULtbJZ36siDm419t_O6yemT0TXf_oimVel9ApbD0CazZUDSkSnmoC2lGBrvB5NOrhZxT8Xn-_k1SaDaN5qj__UZGL2AhNS-XQb8MJZiJp087l4vQqZy1EHP2Su7_OfK2oxDeu67ny/file# [following]
--2023-11-20 19:30:58--  https://ucdaf60311b0d0814212532a2830.dl.dropboxusercontent.com/cd/0/inline/CH5ZVA9DyVq4fWuULtbJZ36siDm419t_O6yemT0TXf_oimVel9ApbD0CazZUDSkSnmoC2lGBrvB5NOrhZxT8Xn-_k1SaDaN5qj__UZGL2AhNS-XQb8MJZiJp087l4vQqZy1EHP2Su7_OfK2oxDeu67ny/file
Resolving ucdaf60311b0d0814212532a2830.dl.dropboxusercontent.com (ucdaf60311b0d0814212532a2830.dl.dropboxusercontent.com)... 162.125.85.15, 2

In [ ]:
!unzip -n /content/CRSP_data.zip

Archive:  /content/CRSP_data.zip


In [ ]:
# Nem tudtam fix width format fájlként beolvasni egyben, hiába próbáltam optimalizálni a beolvasást, nem fért el a memóriában.
# Ebben a cellában átalakítom CSV fájllá. (Mert amúgy nem tsv a fájl, bocs a kiterjesztés miatt!)

input_file = '/content/CRSP_data.tsv'
output_file = '/content/CRSP_data.csv'

i = 1 # az első sorból (header) hiányzik két címke, ezért az első sor elé teszek két szeparátort
with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
    for line in infile:
        modified_line = re.sub(r'\s+', ';', line.strip())
        if i == 1:
          modified_line = ";;" + modified_line
          i = i + 1
        outfile.write(modified_line + '\n')

In [3]:
# Ha az adattisztítást futtatod és már megvan a csv a környezetben, innen futtasd!
crsp = pd.read_csv("/content/CRSP_data.csv", sep=";")
# crsp.head(10)
crsp.shape[0]

<ipython-input-3-8baae3d0eeb8>:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  crsp = pd.read_csv("/content/CRSP_data.csv", sep=";")


40158758

In [4]:
# Adattisztítás
crsp = crsp[crsp.columns[1:]]
crsp.drop('Ticker', inplace=True, axis=1) #ticker nem kell, a permno elég
# Ordinary common share és have not been/not be further defined
crsp.drop(crsp[crsp.SH > 11].index, inplace = True)
crsp.drop(crsp[crsp.EX > 3].index, inplace = True) # NYSE, AMEX, NASDAQ
crsp.drop(crsp[crsp.Ret < -1].index, inplace = True) # Van néhány -100% alatti hozam, azokat törlöm
crsp.loc[crsp['Tvol'] < 0, 'Tvol'] = np.nan # a negatív volumeneket is kiütöm
gc.collect() # memória takarítás
crsp = crsp.rename(columns={'Unnamed: 1': 'date'})
crsp['date'] = pd.to_datetime(crsp['date'], format='%Y%m%d')
crsp.shape[0]

24627594

In [5]:
crsp.head()

,date,PERMNO,CUSIP,SIC,SH,EX,Prc,Tvol,Ret,Shr
0,2000-01-03,10001,36720410,4920,11,3.0,8.5625,1721.0,0.007353,2450.0
1,2000-01-04,10001,36720410,4920,11,3.0,8.4375,1080.0,-0.014599,2450.0
2,2000-01-05,10001,36720410,4920,11,3.0,8.5625,1711.0,0.014815,2450.0
3,2000-01-06,10001,36720410,4920,11,3.0,8.5000,580.0,-0.007299,2450.0
4,2000-01-07,10001,36720410,4920,11,3.0,8.4375,1406.0,-0.007353,2450.0


In [6]:
crsp['AbsPrc'] = crsp['Prc'].abs()
crsp['Size'] = np.log(crsp['AbsPrc'] * crsp['Shr'])
crsp['lnRet'] = np.log(1+crsp['Ret'])
crsp.head()

,date,PERMNO,CUSIP,SIC,SH,EX,Prc,Tvol,Ret,Shr,AbsPrc,Size,lnRet
0,2000-01-03,10001,36720410,4920,11,3.0,8.5625,1721.0,0.007353,2450.0,8.5625,9.951236,0.007326
1,2000-01-04,10001,36720410,4920,11,3.0,8.4375,1080.0,-0.014599,2450.0,8.4375,9.936529,-0.014707
2,2000-01-05,10001,36720410,4920,11,3.0,8.5625,1711.0,0.014815,2450.0,8.5625,9.951236,0.014706
3,2000-01-06,10001,36720410,4920,11,3.0,8.5000,580.0,-0.007299,2450.0,8.5000,9.943909,-0.007326
4,2000-01-07,10001,36720410,4920,11,3.0,8.4375,1406.0,-0.007353,2450.0,8.4375,9.936529,-0.007380


In [ ]:
crsp.to_csv('CRSP_data_ready.csv.zip', compression='zip')

In [9]:
# Az S&P500 forrása - nem túl elengánsan - a Yahoo! Finance, kézzel leszedve.
# A következőkben ezt formázom meg és mentem le, hasonlóan a CRSP adatokhoz.

!wget -O sp500-daily.xlsx https://www.dropbox.com/scl/fi/rqlsdqgw79wvuexru1yhu/sp500-daily.xlsx?rlkey=y9mmou8ey74uy7qegyxrw776y&dl=0

sp500 = pd.read_excel("/content/sp500-daily.xlsx", sheet_name='sp500')
sp500.columns = [x.lower() for x in sp500.columns]
sp500 = sp500.sort_values(by=['date'])
# Dátum
sp500['date'] = pd.to_datetime(sp500['date'], format='%Y-%m-%d %H:%M:%S')
# Napi loghozam
sp500["return_index"] = np.log(sp500["adjclose"] / sp500.shift(1)["adjclose"])
sp500["id"] = 'spx'


--2023-11-20 20:35:55--  https://www.dropbox.com/scl/fi/rqlsdqgw79wvuexru1yhu/sp500-daily.xlsx?rlkey=y9mmou8ey74uy7qegyxrw776y
Resolving www.dropbox.com (www.dropbox.com)... 162.125.85.18, 2620:100:6017:18::a27d:212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.85.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /c/scl/fi/rqlsdqgw79wvuexru1yhu/sp500-daily.xlsx?rlkey=y9mmou8ey74uy7qegyxrw776y [following]
--2023-11-20 20:35:56--  https://www.dropbox.com/c/scl/fi/rqlsdqgw79wvuexru1yhu/sp500-daily.xlsx?rlkey=y9mmou8ey74uy7qegyxrw776y
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucb69fc4fea37a321bb3642d67d9.dl.dropboxusercontent.com/cd/0/get/CH71LZDhtAK0o7QUeZO3jNrvDL9VzOGxZj4QtYbHLV4fS_2P9zslyIm0gDLQThx1hzu-Scd9EbyCvmc2a1tkf6DwUaluvUiula8SEEToOnwOTBc3cjwwksBqJjr2dkfr5UC0n_g9-KElY1EU0tE5Pxil/file# [following]
--2023-11-20 20:35:57--  https://ucb69fc4fea37a321bb3642d67d9.dl.d

In [10]:
sp500.head()

,date,open,high,low,close,adjclose,volume,return_index,id
5754,2001-01-02,1320.28,1320.28,1276.05,1283.27,1283.27,1.129400e+09,NaN,spx
5753,2001-01-03,1283.27,1347.76,1274.62,1347.56,1347.56,1.880700e+09,0.048884,spx
5752,2001-01-04,1347.56,1350.24,1329.14,1333.34,1333.34,2.131000e+09,-0.010608,spx
5751,2001-01-05,1333.34,1334.77,1294.95,1298.35,1298.35,1.430800e+09,-0.026593,spx
5750,2001-01-08,1298.35,1298.35,1276.29,1295.86,1295.86,1.115500e+09,-0.001920,spx


In [11]:
sp500.to_csv('sp500_data_ready.csv.zip', compression='zip')

In [28]:
# És végül az FF3 faktorok hasonlóan

!wget -O F-F_Research_Data_Factors_daily.zip https://www.dropbox.com/scl/fi/p22zek15eha7zh9no2dwx/F-F_Research_Data_Factors_daily_CSV.zip?rlkey=950vi2s9k0y1bvwn379bv3cp3&dl=0
!unzip -n '/content/F-F_Research_Data_Factors_daily.zip'
ff3 = pd.read_csv("/content/F-F_Research_Data_Factors_daily.CSV",
                  sep=",", skiprows=4, skipfooter=3, engine='python')
ff3.head(10)
#ff3.shape[0]

--2023-11-20 20:52:29--  https://www.dropbox.com/scl/fi/p22zek15eha7zh9no2dwx/F-F_Research_Data_Factors_daily_CSV.zip?rlkey=950vi2s9k0y1bvwn379bv3cp3
Resolving www.dropbox.com (www.dropbox.com)... 162.125.85.18, 2620:100:6031:18::a27d:5112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.85.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc486544a9888fb1b9b3e758b824.dl.dropboxusercontent.com/cd/0/inline/CH6CyWYn1Y6GdRbE8R8FgqKwixw84PlnwoWA71WMVCuUdEMc-Go0mRkP191W9MoTEzQtKqf4ZkxPae-I9IR11f52VjnTppiMkIggMD4vRwH_fFxELTTMcfdXElmOd8rJOjTKMneiBfKvfNMnrT8AhLat/file# [following]
--2023-11-20 20:52:30--  https://uc486544a9888fb1b9b3e758b824.dl.dropboxusercontent.com/cd/0/inline/CH6CyWYn1Y6GdRbE8R8FgqKwixw84PlnwoWA71WMVCuUdEMc-Go0mRkP191W9MoTEzQtKqf4ZkxPae-I9IR11f52VjnTppiMkIggMD4vRwH_fFxELTTMcfdXElmOd8rJOjTKMneiBfKvfNMnrT8AhLat/file
Resolving uc486544a9888fb1b9b3e758b824.dl.dropboxusercontent.com (uc486544a9888fb1b9b3e758b824.dl.dropboxuserconten

,Unnamed: 0,Mkt-RF,SMB,HML,RF
0,19260701,0.10,-0.25,-0.27,0.009
1,19260702,0.45,-0.33,-0.06,0.009
2,19260706,0.17,0.30,-0.39,0.009
3,19260707,0.09,-0.58,0.02,0.009
4,19260708,0.21,-0.38,0.19,0.009
5,19260709,-0.71,0.43,0.57,0.009
6,19260710,0.62,-0.53,-0.10,0.009
7,19260712,0.04,-0.03,0.64,0.009
8,19260713,0.48,-0.28,-0.20,0.009
9,19260714,0.04,0.07,-0.43,0.009


In [29]:
ff3 = ff3.rename(columns={'Unnamed: 0': 'date'})
ff3['date'] = pd.to_datetime(ff3['date'], format='%Y%m%d')
ff3 = ff3[ff3['date'] >= '200001-03']
ff3.head()

,date,Mkt-RF,SMB,HML,RF
19865,2001-01-03,5.39,0.71,-4.33,0.026
19866,2001-01-04,-1.30,1.03,-0.25,0.026
19867,2001-01-05,-2.98,0.15,2.38,0.026
19868,2001-01-08,-0.36,-0.68,1.25,0.026
19869,2001-01-09,0.51,0.58,-0.39,0.026


In [34]:
#ff3.dtypes
ff3.to_csv('ff3_data_ready.csv.zip', compression='zip')